In [9]:
from Adafruit_IO import Client, Feed
from i2c_sensor_classes import Ky015Sensor, Ky018Sensor
from aio_helper_classes import HabitatMonitorClient
import time

In [10]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
base = BaseOverlay("base.bit")

# Testing Reading from the Sensors 
no cloud here, trying to decouple cloud code and sensor reading code

In [11]:
KY015_ADDR = 8
KY018_ADDR = 0x28
SCL_PIN = 2
SDA_PIN = 3

liba = MicroblazeLibrary(base.PMODA, ['i2c'])
#custom class see 'src/i2c_sensor_classes.py'
ky015 = Ky015Sensor(liba, SDA_PIN, SCL_PIN, KY015_ADDR)

libb = MicroblazeLibrary(base.PMODB, ['i2c'])
#custom class see 'src/i2c_sensor_classes.py'
ky018 = Ky018Sensor(libb, SDA_PIN, SCL_PIN, KY018_ADDR)

response = input('Read Data?(y/n): ')
while response != 'n':
    temp, hum = ky015.get_temp_and_hum_data()
    bright = ky018.get_brightness_data()
    print('\tTemp: {}C, Hum: {}%\n\tBright: {}'.format(temp,hum, bright))
    response = input('Read Data?(y/n): ')

Read Data?(y/n): y
	Temp: 20.06C, Hum: 44.0%
	Bright: 252
Read Data?(y/n): y
	Temp: 20.06C, Hum: 44.0%
	Bright: 248
Read Data?(y/n): y
	Temp: 20.06C, Hum: 44.0%
	Bright: 244
Read Data?(y/n): n


# Testing the Habitat Monitor class
this class extends the default Adafruit_IO Client, with functionality specific to out Habitat Monitor. If check the client_id and either sets the feeds to the correct Feed Objects, or if the client_id is new then new data feeds are created and associated with the passed client-id. 

In [ ]:
UNAME = input('Enter username: ')
KEY = input('Enter Key: ')
client_id = 'joeys-pynq-board'

#custom class see "src/aio_helper_classes.py"
aio = HabitatMonitorClient(UNAME, KEY, client_id)    

In [ ]:
response = 'y'
while response != 'n':
    response = input('Read Data?(y/n): ')
    if response == 'n': break
    temp, hum = ky015.get_temp_and_hum_data()
    print('\tTemp: {}C, Hum: {}%'.format(temp,hum))
    response = input('Publish to Cloud?(y/n): ')
    if response == 'n': break
    aio.send(aio.temperature_feed.key, temp)
    aio.send(aio.humidity_feed.key, hum)
    print('\tData Sent to Adafrui.io')